In [ ]:
import torch
import os
import pandas as pd

from torch.utils.data import Dataset, DataLoader
import torchvision.models as models

class Data(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

batch_size = 64

train_data = torch.load("/home/walke/college/cv/ass2/CV Assignment 2/Q1/train_data.pt")
train_labels = torch.load("/home/walke/college/cv/ass2/CV Assignment 2/Q1/train_labels.pt")

val_data = train_data[:10000]
val_labels = train_labels[:10000]

train_data = train_data[10000:]
train_labels = train_labels[10000:]


train_dataset = Data(train_data, train_labels)
val_dataset = Data(val_data, val_labels)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)


test_data = torch.load("/home/walke/college/cv/ass2/CV Assignment 2/Q1/test_data.pt")
test_labels = torch.load("/home/walke/college/cv/ass2/CV Assignment 2/Q1/test_labels.pt")

test_dataset = Data(test_data, test_labels)

test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

model = models.resnet18()
model.fc = torch.nn.Linear(model.fc.in_features, 10)

model.to(device)

from tqdm import tqdm

def train_loop(dataloader, model, loss_fn, optimizer):
    total = len(dataloader)
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, correct = 0, 0

    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in tqdm(enumerate(dataloader),desc="train", total = total):
        # Compute prediction and loss
        optimizer.zero_grad()
        
        X, y = X.to(device).float(), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        train_loss += loss.item()

        # Backpropagation
        loss.backward()
        optimizer.step()
        

        correct += (pred.argmax(1) == y).type(torch.float).sum().item()


        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)


    train_loss /= num_batches
    correct /= size
    print(f"Training Error: \n Accuracy: {(100*correct):>0.1f}%, Avg Training loss: {train_loss:>8f} \n")
    

def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    # model.load_state_dict(torch.load(model_path))

    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device).float(), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            
            # _, y = torch.argmax(y, dim=1)  
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


def val_loop(dataloader, model, loss_fn):

    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    val_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device).float(), y.to(device)
            pred = model(X)
            val_loss += loss_fn(pred, y).item()
            
            # _, y = torch.argmax(y, dim=1)  
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    val_loss /= num_batches
    correct /= size
    print(f"Validation Error: \n Accuracy: {(100*correct):>0.1f}%, Avg Validation loss: {val_loss:>8f} \n")

    return correct

learning_rate = 1e-3
batch_size = 64
epochs = 10

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# model.to(device)
val_accuracy_highest = 0
val_accuracy_curr = 0

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    val_accuracy_curr = val_loop(val_dataloader, model, loss_fn)
    
    # if val_accuracy_curr > val_accuracy_highest:
    #     val_accuracy_highest = val_accuracy_curr
    #     model_path = 'model_1'
    #     print(f"Saving Model at epoch: {t+1}\n")
    #     torch.save(model.state_dict(), model_path)

    
print("Done!")

test_loop(test_dataloader, model, loss_fn)

/tmp/ipykernel_510689/634928451.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load("/home/walke/college/cv/ass2/CV Assignment 2/Q1/train_data.pt")


In [ ]:
from torchvision.models import resnet18, ResNet18_Weights

weights = ResNet18_Weights.DEFAULT
model_trained = resnet18(weights=weights)

for param in model_trained.parameters():
    param.requires_grad = False

model_trained.fc = torch.nn.Linear(model_trained.fc.in_features, 10) 

model_trained.to(device)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 10

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_trained.parameters(), lr=learning_rate)

# model_trained.to(device)
val_accuracy_highest = 0
val_accuracy_curr = 0

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model_trained, loss_fn, optimizer)
    val_accuracy_curr = val_loop(val_dataloader, model_trained, loss_fn)
    
    # if val_accuracy_curr > val_accuracy_highest:
    #     val_accuracy_highest = val_accuracy_curr
    #     model_trained_path = 'model_trained_1'
    #     print(f"Saving Model_trained at epoch: {t+1}\n")
    #     torch.save(model_trained.state_dict(), model_trained_path)

    
print("Done!")

test_loop(test_dataloader, model_trained, loss_fn)



Epoch 1
-------------------------------


train: 100%|██████████| 625/625 [00:11<00:00, 52.15it/s]


Training Error: 
 Accuracy: 69.0%, Avg Training loss: 0.917968 

Validation Error: 
 Accuracy: 71.4%, Avg Validation loss: 0.868923 

Epoch 2
-------------------------------


train: 100%|██████████| 625/625 [00:14<00:00, 42.36it/s]


Training Error: 
 Accuracy: 79.1%, Avg Training loss: 0.618760 

Validation Error: 
 Accuracy: 74.4%, Avg Validation loss: 0.759513 

Epoch 3
-------------------------------


...

Training Error: 
 Accuracy: 94.5%, Avg Training loss: 0.160902 

Validation Error: 
 Accuracy: 79.8%, Avg Validation loss: 0.782511 

Epoch 9
-------------------------------


train: 100%|██████████| 625/625 [00:14<00:00, 43.63it/s]


Training Error: 
 Accuracy: 94.9%, Avg Training loss: 0.151246 

Validation Error: 
 Accuracy: 79.3%, Avg Validation loss: 0.835215 

Epoch 10
-------------------------------


train: 100%|██████████| 625/625 [00:11<00:00, 53.68it/s]


Training Error: 
 Accuracy: 96.2%, Avg Training loss: 0.110765 

Validation Error: 
 Accuracy: 79.8%, Avg Validation loss: 0.797384 

Done!
Test Error: 
 Accuracy: 79.1%, Avg loss: 0.827922 

